In [1]:
import pandas as pd
import polars as pl
import numpy as np
import os
import gc
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import KFold, StratifiedKFold
import xgboost as xgb
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor, log_evaluation, record_evaluation
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
#from sklearn.impute import IterativeImputer
import pickle
import optuna
from optuna.visualization import plot_slice, plot_param_importances
import shap
import random

gc.enable()

pd.options.display.max_columns = None
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

pl.Config.set_tbl_rows(-1)
pl.Config.set_tbl_cols(-1)
pl.Config.set_fmt_str_lengths(10000)
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [2]:
path = 'I:/Kaggle/jane-street-real-time-market-data-forecasting/'

In [3]:
os.listdir(path)

['features.csv',
 'kaggle_evaluation',
 'lags.parquet',
 'my_folder',
 'responders.csv',
 'sample_submission.csv',
 'team_folder',
 'test.parquet',
 'top_100000_rows_sorted_by_weight_descending.parquet',
 'top_10000_rows_sorted_by_weight_descending.parquet',
 'train.parquet']

In [4]:
train_df = pl.read_parquet(path + 'train.parquet/').drop(['responder_0', 'responder_1', 'responder_2', 'responder_3', 'responder_4', 'responder_5', 'responder_7', 'responder_8', 'partition_id']).select(pl.all().shrink_dtype())
print(train_df.shape)
train_df.head()

(47127338, 84)


date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_6
i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,0,1,3.889038,null,null,null,null,null,0.851033,0.242971,0.2634,-0.891687,11,7,76,-0.883028,0.003067,-0.744703,null,-0.169586,null,-1.335938,-1.707803,0.91013,null,1.636431,1.522133,-1.551398,-0.229627,null,null,1.378301,-0.283712,0.123196,null,null,null,0.28118,0.269163,0.349028,-0.012596,-0.225932,null,-1.073602,null,null,-0.181716,null,null,null,0.564021,2.088506,0.832022,null,0.204797,null,null,-0.808103,null,-2.037683,0.727661,null,-0.989118,-0.345213,-1.36224,null,null,null,null,null,-1.251104,-0.110252,-0.491157,-1.02269,0.152241,-0.659864,null,null,-0.261412,-0.211486,-0.335556,-0.281498,0.775981
0,0,7,1.370613,null,null,null,null,null,0.676961,0.151984,0.192465,-0.521729,11,7,76,-0.865307,-0.225629,-0.582163,null,0.317467,null,-1.250016,-1.682929,1.412757,null,0.520378,0.744132,-0.788658,0.641776,null,null,0.2272,0.580907,1.128879,null,null,null,-1.512286,-1.414357,-1.823322,-0.082763,-0.184119,null,null,null,null,null,null,null,null,-10.835207,-0.002704,-0.621836,null,1.172836,null,null,-1.625862,null,-1.410017,1.063013,null,0.888355,0.467994,-1.36224,null,null,null,null,null,-1.065759,0.013322,-0.592855,-1.052685,-0.393726,-0.741603,null,null,-0.281207,-0.182894,-0.245565,-0.302441,0.703665
0,0,9,2.285698,null,null,null,null,null,1.056285,0.187227,0.249901,-0.77305,11,7,76,-0.675719,-0.199404,-0.586798,null,-0.814909,null,-1.296782,-2.040234,0.639589,null,1.597359,0.657514,-1.350148,0.364215,null,null,-0.017751,-0.317361,-0.122379,null,null,null,-0.320921,-0.95809,-2.436589,0.070999,-0.245239,null,null,null,null,null,null,null,null,-1.420632,-3.515137,-4.67776,null,0.535897,null,null,-0.72542,null,-2.29417,1.764551,null,-0.120789,-0.063458,-1.36224,null,null,null,null,null,-0.882604,-0.072482,-0.617934,-0.86323,-0.241892,-0.709919,null,null,0.377131,0.300724,-0.106842,-0.096792,2.109352
0,0,10,0.690606,null,null,null,null,null,1.139366,0.273328,0.306549,-1.262223,42,5,150,-0.694008,3.004091,0.114809,null,-0.251882,null,-1.902009,-0.979447,0.241165,null,-0.392359,-0.224699,-2.129397,-0.855287,null,null,0.404142,-0.578156,0.105702,null,null,null,0.544138,-0.087091,-1.500147,-0.201288,-0.038042,null,null,null,null,null,null,null,null,0.382074,2.669135,0.611711,null,2.413415,null,null,1.313203,null,-0.810125,2.939022,null,3.988801,1.834661,-1.36224,null,null,null,null,null,-0.697595,1.074309,-0.206929,-0.530602,4.765215,0.571554,null,null,-0.226891,-0.251412,-0.215522,-0.296244,1.114137
0,0,14,0.44057,null,null,null,null,null,0.9552,0.262404,0.344457,-0.613813,44,3,16,-0.947351,-0.030018,-0.502379,null,0.646086,null,-1.844685,-1.58656,-0.182024,null,-0.969949,-0.673813,-1.282132,-1.399894,null,null,0.043815,-0.320225,-0.031713,null,null,null,-0.08842,-0.995003,-2.635336,-0.196461,-0.618719,null,n

In [5]:
train_df.estimated_size() / 1e9

15.282841602

In [6]:
previous_models_path = path + 'my_folder/models/20250107_02/'

In [7]:
models_path = path + 'my_folder/models/20250107_03/'
if not os.path.exists(models_path):
    os.makedirs(models_path)

In [8]:
lgb_params_df = pd.read_csv(previous_models_path + 'lgb_params.csv')
print(lgb_params_df.shape)
lgb_params_df

(1, 9)


,window_size,learning_rate,max_depth,min_data_in_leaf,num_leaves,min_gain_to_split,lambda_l1,lambda_l2,feature_fraction
0,8294329,0.043114,15,154,3047,0.277803,7.028187,897.533632,0.818306


In [9]:
def lgb_sliding_window(train_data, params_df):
    unique_date_ids = sorted(train_data['date_id'].unique())
    date_ids_df = train_data['date_id'].to_frame()
    window_size = params_df['window_size'][0]

    for date_id in tqdm(unique_date_ids):    
        date_id_df = train_data.filter(pl.col('date_id') == date_id)
        if date_ids_df.filter(pl.col('date_id') < date_id).shape[0] < window_size:            
            window_df = train_data[:window_size + date_id_df.shape[0]]
            window_df = window_df.join(date_id_df, on=['date_id', 'time_id', 'symbol_id'], how='anti')
        else:
            window_df = train_data.filter(pl.col('date_id') < date_id)[-window_size:]

        window_df = window_df.sample(fraction=0.1)

        print('date_id:', date_id)
    
        base_params = {
            'verbosity': -1,
            'device': 'gpu',
            'early_stopping_round': 20,
        }
    
        tuned_params = {
            'learning_rate': params_df['learning_rate'][0],
            'max_depth': params_df['max_depth'][0],
            'min_data_in_leaf': params_df['min_data_in_leaf'][0],
            'num_leaves': params_df['num_leaves'][0],
            'min_gain_to_split': params_df['min_gain_to_split'][0],
            'lambda_l1': params_df['lambda_l1'][0],
            'lambda_l2': params_df['lambda_l2'][0],
            'feature_fraction': params_df['feature_fraction'][0],
        }
    
        model = LGBMRegressor(
            **base_params,
            **tuned_params,
            n_estimators=100000
        )
    
        X_train = window_df.drop(['date_id', 'time_id', 'symbol_id', 'weight', 'responder_6']).select(pl.all().shrink_dtype()).to_pandas()
        X_val = date_id_df.drop(['date_id', 'time_id', 'symbol_id', 'weight', 'responder_6']).select(pl.all().shrink_dtype()).to_pandas()
    
        y_train = window_df['responder_6'].to_pandas()
        y_val = date_id_df['responder_6'].to_pandas()
    
        weights_train = window_df['weight'].to_pandas()
        weights_val = date_id_df['weight'].to_pandas()
    
        model.fit(X_train, y_train, sample_weight=weights_train, eval_set=[(X_train, y_train), (X_val, y_val)], eval_sample_weight=[weights_train, weights_val], callbacks=[log_evaluation(period=10)])

        val_preds = model.predict(X_val)
        val_r2_score = r2_score(y_val, val_preds, sample_weight=weights_val)

        print('Val Weighted R2 score is:', val_r2_score)

In [10]:
lgb_sliding_window(train_df, lgb_params_df)

  0%|                                                                                         | 0/1699 [00:00<?, ?it/s]

date_id: 0
[10]	training's l2: 0.760208	valid_1's l2: 0.226788
[20]	training's l2: 0.748475	valid_1's l2: 0.226195
[30]	training's l2: 0.738287	valid_1's l2: 0.226016
[40]	training's l2: 0.729745	valid_1's l2: 0.225777
[50]	training's l2: 0.722627	valid_1's l2: 0.225671
[60]	training's l2: 0.716782	valid_1's l2: 0.225751


  0%|                                                                               | 1/1699 [00:21<9:54:54, 21.02s/it]

Val Weighted R2 score is: 0.012053470181566328
date_id: 1
[10]	training's l2: 0.750082	valid_1's l2: 0.644241
[20]	training's l2: 0.738653	valid_1's l2: 0.640232
[30]	training's l2: 0.728643	valid_1's l2: 0.636843
[40]	training's l2: 0.720229	valid_1's l2: 0.635085
[50]	training's l2: 0.713199	valid_1's l2: 0.633997
[60]	training's l2: 0.708167	valid_1's l2: 0.633861
[70]	training's l2: 0.704375	valid_1's l2: 0.63371
[80]	training's l2: 0.701035	valid_1's l2: 0.633168
[90]	training's l2: 0.697802	valid_1's l2: 0.63334
[100]	training's l2: 0.694546	valid_1's l2: 0.632709
[110]	training's l2: 0.692199	valid_1's l2: 0.633171
[120]	training's l2: 0.689002	valid_1's l2: 0.632466
[130]	training's l2: 0.686376	valid_1's l2: 0.63191
[140]	training's l2: 0.683917	valid_1's l2: 0.631974
[150]	training's l2: 0.681	valid_1's l2: 0.632247


  0%|                                                                              | 2/1699 [00:49<12:00:42, 25.48s/it]

...

Val Weighted R2 score is: 0.050345922342013294
date_id: 111
[10]	training's l2: 0.750675	valid_1's l2: 0.662869
[20]	training's l2: 0.738911	valid_1's l2: 0.659819
[30]	training's l2: 0.728502	valid_1's l2: 0.658183
[40]	training's l2: 0.719703	valid_1's l2: 0.657322
[50]	training's l2: 0.711953	valid_1's l2: 0.656892
[60]	training's l2: 0.705689	valid_1's l2: 0.656847
[70]	training's l2: 0.700752	valid_1's l2: 0.656336
[80]	training's l2: 0.697149	valid_1's l2: 0.656371
[90]	training's l2: 0.69392	valid_1's l2: 0.65652


  7%|█████                                                                       | 112/1699 [51:50<14:43:28, 33.40s/it]

Val Weighted R2 score is: 0.004702165456882956
date_id: 112
[10]	training's l2: 0.753431	valid_1's l2: 0.501164
[20]	training's l2: 0.741664	valid_1's l2: 0.497766
[30]	training's l2: 0.731509	valid_1's l2: 0.495617


  7%|█████                                                                       | 112/1699 [52:06<12:18:23, 27.92s/it]


KeyboardInterrupt: 